# COMPARACIÓN DEL Nº DE COPIA DE LOS GENES COMUNES
Este script es parte del 3_copy_num, pero lo separo pq va a ser mucho código de golpe, muy distinto, y me ayuda tener las cosas cada una por su cuenta
OBJETIVO: comprobar si hay diferencias significativas en el nº de copias de ARG y MGE INDIVIDUALES, COMPARTIDOS entre plásticos y control. De TODOS, ZONA DE MUESTREO A ZONA DE MUESTREO (¡necesariamente tengo que trabajar con los datos chorizeados del Ardley y con Ion + Uruguay juntos!)

Ojo que hay dos comparaciones distintas aquí a considerar:
1. Comparar el NCN de ARG/MGE considerando TODAS las muestras (= volver a meter 0s)
2. Comparar el NCN de ARG/MGE considerando SÓLO cuando están presentes (= no meter 0s de vuelta)

De la primera manera, el hecho de que pueda haber más detecciones per se en plástico afectará. De la segunda, la pregunta es más específica y las conclusiones menos extrapolables, pero es necesario complementar la primera respuesta con la segunda.

---

Y sea cual sea el caso, lo primero siempre va a ser ver qué genes son comunes en plástico y medio circundante muestra a muestra. Ese código, ese listado lo puedo reciclar, y usarlo por igual en ambas aproximaciones.

In [1]:
import pandas as pd
import scipy.stats as stats
import numpy as np
import scikit_posthocs as sp
import matplotlib
import warnings
warnings.filterwarnings("ignore")

## COMPARACIÓN DE NCN A NIVEL DE GEN INDIVIDUAL CONSIDERANDO TODOS LOS RESULTADOS

In [2]:
all_data = pd.read_csv("../data/resistome_data/clean_data/ab_data_all_fixed.csv", index_col = 0) # CHORICEO
all_data.dropna(inplace = True)
all_data = all_data[(all_data["antib"] != "16S")]
all_data = all_data[["Assay", "antib", "sample", "place", "type_f", "type_g", "or_seq", "rel_ab"]] #place viene a ser la estrella del show
all_data = all_data.loc[~all_data["place"].isin([0])]
all_data.loc[all_data["sample"] == 13, "rel_ab"] = all_data.loc[all_data["sample"] == 13]["rel_ab"] / 3
all_data.loc[all_data["sample"] == 14, "rel_ab"] = all_data.loc[all_data["sample"] == 14]["rel_ab"] / 3
#all_data = all_data[(all_data.Assay != "aadA7") & (all_data.sample != 5)] #esto también es tremendamente importante

common_amps = all_data[["Assay", "place", "type_g"]].groupby(["place", "Assay"])["type_g"].unique()
del all_data
common_amps = common_amps.reset_index()
common_amps["type_g"] = common_amps["type_g"].astype("str") #hacky as shit
common_amps = common_amps.loc[(common_amps["type_g"] == "['plastic' 'control']") | (common_amps["type_g"] == "['control' 'plastic']")]
common_amps

,place,Assay,type_g
0,ardley,ARR-3,['plastic' 'control']
1,ardley,Aac6-Aph2,['plastic' 'control']
3,ardley,Arr2,['plastic' 'control']
6,ardley,IS1133,['plastic' 'control']
7,ardley,IS1247,['plastic' 'control']
...,...,...,...
422,uru,mphA,['control' 'plastic']
423,uru,oprD,['plastic' 'control']
424,uru,orf37-IS26,['plastic' 'control']
426,uru,qepA_1_2,['plastic' 'control']


Ahora procedo a quedarme sólo con las partes específicas en las que me interesa estudiar la diferencia. De entrada, separaré lagos y Ardley por la gran influencia de este sobre todos los resultados

In [3]:
# Voy a 0s, endo me quedo todos los ceros habidos y por haber
all_data = pd.read_csv("../data/resistome_data/clean_data/ab_data_all_fixed.csv", index_col = 0) # CHORICEO
all_data = all_data[["Assay", "sample", "place", "type_f", "or_seq", "rel_ab"]]
all_data["rel_ab"].fillna(0, inplace = True)
all_data["n_log2"] = np.log2(all_data["rel_ab"] + 1)
#print(all_data.shape)
#all_data.loc[(all_data["place"] == "ion") |(all_data["place"] == "uru"), "place"] = "lake"
lake_data = all_data.loc[(all_data["place"] == "ion") |(all_data["place"] == "uru")] #sólo los lagos
temp = lake_data.shape[0]
lake_data = lake_data.loc[((lake_data["place"] == "ion") & (lake_data["Assay"].isin(list(common_amps.loc[common_amps["place"] == "ion"]["Assay"].unique()))))
                         |
                         ((lake_data["place"] == "uru") & (lake_data["Assay"].isin(list(common_amps.loc[common_amps["place"] == "uru"]["Assay"].unique()))))]
print("There are ", all_data.shape[0], "ARGvMGE in all the data. ", temp, " just in lake samples. ", lake_data.shape[0], "are shared among plastics and the environment (with all replicates)")
print("(which correspond to ", len(lake_data["Assay"].unique()), " unique sequences)")
lake_data.loc[(lake_data["place"] == "ion") |(lake_data["place"] == "uru"), "place"] = "lake"
ardley_data = all_data.loc[all_data["place"] == "ardley"]
temp = ardley_data.shape[0]
ardley_data = ardley_data.loc[ardley_data["Assay"].isin(list(common_amps.loc[common_amps["place"] == "ardley"]["Assay"].unique()))]
print("There are ", all_data.shape[0], "ARGvMGE in all the data. ", temp, " just in ardley samples. ", ardley_data.shape[0], "are shared among plastics and the environment (with all replicates)")
print("(which correspond to ", len(ardley_data["Assay"].unique()), " unique sequences)")
del all_data
del temp

There are  10314 ARGvMGE in all the data.  6876  just in lake samples.  246 are shared among plastics and the environment (with all replicates)
(which correspond to  20  unique sequences)
There are  10314 ARGvMGE in all the data.  3438  just in ardley samples.  2115 are shared among plastics and the environment (with all replicates)
(which correspond to  235  unique sequences)


In [4]:
count = 0
for dataset in [lake_data, ardley_data]:
    for seq in dataset["Assay"].unique():
        wd = dataset.loc[dataset["Assay"] == seq] #working data: nuestro Assay de interés en cada momento
        var_lst_ln = [group["n_log2"] for name, group in wd.groupby(["or_seq", "type_f"])]
        nam_lst = [name for name, group in wd.groupby(["or_seq", "type_f"])]
        levene = stats.levene(*var_lst_ln)
        if levene[1] >= 0.05:
            kruskal = stats.kruskal(*var_lst_ln)
            if kruskal[1] < 0.05: 
                count += 1
                #print("Results for sequence ", seq, "in ", wd["place"].unique())
                #print(kruskal)
                #post_lolo = sp.posthoc_conover(var_lst_ln, p_adjust = "fdr_bh")
                #post_lolo.rename(columns = {original_num:new_name for (original_num, new_name) in zip(post_lolo.columns, nam_lst)},
                #                 index = {original_num:new_name for (original_num, new_name) in zip(post_lolo.index, nam_lst)},
                #                 inplace = True)
                #post_lolo = post_lolo[post_lolo < 0.05]
                #print(post_lolo)

print(count)            
# Y ahora ver solo lo interesante
#post_lolo = post_lolo[post_lolo < 0.05]
#esto enfrenta a control (filas) vs plástico (columnas)
#post_lolo_control = post_lolo.iloc[0:post_lolo.shape[0]:2,1:(post_lolo.shape[1]+1):2]
#y esto saca sólo la diagonal
#post_lolo_diag = pd.Series(np.diag(post_lolo_control), index=[post_lolo_control.index, post_lolo_control.columns])
#post_lolo

68


Ahora voy a comparar por pares plástico vs control sin más remordimiento. Esto va a suponer dos problemas inmediatos:
1. Si hago un test estadístico con un valor = NaN, el resultado no va a ser fiable (va a ser NaN), puesto que estoy metiendo infinitos
2. Paralelamente, si no tengo tres réplicas, el test no corre. Asi que no puedo usar todo el dataset.

In [5]:
def statistic_mean(x, y, axis):
    return np.mean(x, axis=axis) - np.mean(y, axis=axis)
def get_difs(groupby_data, var = "rel_ab", detailed = False, alternative = "two-sided", significant = True, grouping_var = "type_g"):
    """
    A highly hardcoded function that I'm defining only because it will make the rest of the notebook cleaner. Designed to take a multi-indexed groupby object
    and perform some basic analysys on it: normality, homocedasticity and the corresponding test of means' difference. It probably incurs in Type I error, as it compares multiples
    pairs of control-sample samples (they are NOT paired samples, mind you)
    
    - groupby_data: the groupby object that contains the data to analyze. As it is programmed, it MUST contain a column with a descriptive, binary value: control and study samples
    - var: name of the column whose values we want to compare
    - detailed: change it to True to get a detailed printing of all the tests being conducted
    - alternative: which altenative hypothesis to feed to the paired tests. I0m running scipy tests, so it works with the same values (less, two-sided, greater)
    - significant: if True, keep just those with pval < 0.05; if False, keep everything
    """
    result_dict = {}
    for name, group in groupby_data:
        name = str(name).upper()
        pl_data = group.loc[group[grouping_var] == "plastic"][var]
        con_data = group.loc[group[grouping_var] == "control"][var]
        
        # Tests de normalidad
        norm_plas = stats.shapiro(pl_data)
        norm_con = stats.shapiro(con_data)
        
        # Tests de homocedasticidad
        if (norm_plas[1] > 0.05) and (norm_con[1] > 0.05):
            norm = True
            homo = stats.bartlett(pl_data, con_data)
        else: 
            norm = False
            homo = stats.levene(pl_data, con_data)
            
        # Test de igualdad de distribuciones
        if (homo[1] > 0.05) and (norm == True): 
            result = stats.ttest_ind(a = pl_data, b = con_data, alternative = alternative)
        elif (homo[1] > 0.05): #homocedásticos no normales
            result = stats.mannwhitneyu(x = pl_data, y = con_data, alternative = alternative)
        elif (homo[1] < 0.05) and (norm == True): #normales no homocedásticos
            result = stats.ttest_ind(a = pl_data, b = con_data, alternative = alternative, equal_var = False)
            result = ("Welch!", result[1], result[0])
        else: #todo mal, permuta
            result = stats.permutation_test(data = (pl_data, con_data), statistic =  statistic_mean, 
                                            vectorized = True, n_resamples = 20000,
                                            alternative = alternative, permutation_type = "independent")
            result = ("Permutation", result.pvalue, result.statistic) #ugly hack!
        if significant == True:
            if result[1] < 0.05: result_dict[name] = result
        else: 
            result_dict[name] = result
            
            
            
        if detailed == True:
            print("##########", name, "#########")
            print("### PLASTIC ###")
            print(norm_plas)
            print("### CONTROL ###")
            print(norm_con)
            print("### BOTH ###")
            print(homo)
            print("### DIFFERENCES ###")
            print(result)
            print("")
    return result_dict

In [6]:
def get_difs(groupby_data, var = "rel_ab", detailed = False, alternative = "two-sided", significant = True, grouping_var = "type_g"):
    """
    A highly hardcoded function that I'm defining only because it will make the rest of the notebook cleaner. Designed to take a multi-indexed groupby object
    and perform some basic analysys on it: normality, homocedasticity and the corresponding test of means' difference. It probably incurs in Type I error, as it compares multiples
    pairs of control-sample samples (they are NOT paired samples, mind you)
    
    - groupby_data: the groupby object that contains the data to analyze. As it is programmed, it MUST contain a column with a descriptive, binary value: control and study samples
    - var: name of the column whose values we want to compare
    - detailed: change it to True to get a detailed printing of all the tests being conducted
    - alternative: which altenative hypothesis to feed to the paired tests. I0m running scipy tests, so it works with the same values (less, two-sided, greater)
    - significant: if True, keep just those with pval < 0.05; if False, keep everything
    """
    result_dict = {}
    for name, group in groupby_data:
        name = str(name)#.upper()
        pl_data = group.loc[group[grouping_var] == "plastic"][var]
        con_data = group.loc[group[grouping_var] == "control"][var]
        
        # Tests de normalidad
        norm_plas = stats.shapiro(pl_data)
        norm_con = stats.shapiro(con_data)
        
        # Tests de homocedasticidad
        if (norm_plas[1] > 0.05) and (norm_con[1] > 0.05):
            norm = True
            homo = stats.bartlett(pl_data, con_data)
        else: 
            norm = False
            homo = stats.levene(pl_data, con_data)
            
        # Test de igualdad de distribuciones
        if (homo[1] > 0.05) and (norm == True): 
            result = stats.ttest_ind(a = pl_data, b = con_data, alternative = alternative)
        elif (homo[1] > 0.05): #homocedásticos no normales
            result = stats.mannwhitneyu(x = pl_data, y = con_data, alternative = alternative)
        elif (homo[1] < 0.05) and (norm == True): #normales no homocedásticos
            result = stats.ttest_ind(a = pl_data, b = con_data, alternative = alternative, equal_var = False)
            result = ("Welch!", result[1], result[0])
        else: #todo mal, permuta
            result = stats.permutation_test(data = (pl_data, con_data), statistic =  statistic_mean, 
                                            vectorized = True, n_resamples = 20000,
                                            alternative = alternative, permutation_type = "independent")
            result = ("Permutation", result.pvalue, result.statistic) #ugly hack!
        if significant == True:
            if result[1] < 0.05: result_dict[name] = result
        else: 
            result_dict[name] = result
            
            
            
        if detailed == True:
            print("##########", name, "#########")
            print("### PLASTIC ###")
            print(norm_plas)
            print("### CONTROL ###")
            print(norm_con)
            print("### BOTH ###")
            print(homo)
            print("### DIFFERENCES ###")
            print(result)
            print("")
    return result_dict

In [7]:
all_data = pd.read_csv("../data/resistome_data/clean_data/ab_data_all_fixed.csv", index_col = 0)
all_data = all_data[["Assay", "sample", "place", "type_g", "or_seq", "rel_ab"]]
all_data["rel_ab"].fillna(0, inplace = True)
all_data["n_log2"] = np.log2(all_data["rel_ab"] + 1)  #esto evita meter 0s
all_data.loc[all_data["sample"] == 13, "rel_ab"] = all_data.loc[all_data["sample"] == 13]["rel_ab"] / 3
all_data.loc[all_data["sample"] == 14, "rel_ab"] = all_data.loc[all_data["sample"] == 14]["rel_ab"] / 3
all_data = all_data[(all_data.Assay != "aadA7") & (all_data.sample != 5)] #esto también es tremendamente importante
#all_data.dropna(inplace = True)

#print(all_data.shape)
#all_data.loc[(all_data["place"] == "ion") |(all_data["place"] == "uru"), "place"] = "lake"
lake_data = all_data.loc[(all_data["place"] == "ion") |(all_data["place"] == "uru")] #sólo los lagos
temp = lake_data.shape[0]
lake_data = lake_data.loc[((lake_data["place"] == "uru") & (lake_data["Assay"].isin(list(common_amps.loc[common_amps["place"] == "uru"]["Assay"].unique()))))]
print("There are ", all_data.shape[0], "ARGvMGE in all the data. ", temp, " just in lake samples. ", lake_data.shape[0], "are shared among plastics and the environment (with all replicates)")
print("(which correspond to ", len(lake_data["Assay"].unique()), " unique sequences)")
lake_data.loc[(lake_data["place"] == "ion") |(lake_data["place"] == "uru"), "place"] = "lake"
ardley_data = all_data.loc[all_data["place"] == "ardley"]
temp = ardley_data.shape[0]
ardley_data = ardley_data.loc[ardley_data["Assay"].isin(list(common_amps.loc[common_amps["place"] == "ardley"]["Assay"].unique()))]
print("There are ", all_data.shape[0], "ARGvMGE in all the data. ", temp, " just in ardley samples. ", ardley_data.shape[0], "are shared among plastics and the environment (with all replicates)")
print("(which correspond to ", len(ardley_data["Assay"].unique()), " unique sequences)")
del all_data
del temp

There are  10287 ARGvMGE in all the data.  6858  just in lake samples.  168 are shared among plastics and the environment (with all replicates)
(which correspond to  14  unique sequences)
There are  10287 ARGvMGE in all the data.  3429  just in ardley samples.  2106 are shared among plastics and the environment (with all replicates)
(which correspond to  234  unique sequences)


In [8]:
general_log2_difs = {}
general_log2_difs["lake_difs"] = get_difs(lake_data.groupby("Assay"), var = "n_log2",
                                           detailed = False, alternative = "two-sided", significant = True)
general_log2_difs["ardley_difs"] = get_difs(ardley_data.groupby("Assay"), var = "n_log2",
                                           detailed = False, alternative = "two-sided", significant = True)
#general_log2_difs

Ahora tengo un listado de todos las secuencias que tienen una diferencia estadísticamente significativa entre control y plástico, pero no sé si todos todos tienen mayor NCN en plásticos y no hay alguno travieso en control. Asi que voy a hacer algo estúpido: loopear por las llaves del diccionario de resultados, sacar todos los datos de esa secuencia para n_log2 y restar la suma de todos los plásticos a la suma de todos los controles. Si saliera negativo, es mayor en lagos

In [9]:
for seq in general_log2_difs["lake_difs"].keys():
    interest_data = lake_data.loc[lake_data["Assay"] == seq]
    dif = interest_data.loc[interest_data["type_g"] == "plastic"]["n_log2"].sum() - interest_data.loc[interest_data["type_g"] == "control"]["n_log2"].sum()
    if dif > 0:
        general_log2_difs["lake_difs"][seq] = list(general_log2_difs["lake_difs"][seq])
        general_log2_difs["lake_difs"][seq].append("plastic")
    else:
        general_log2_difs["lake_difs"][seq] = list(general_log2_difs["lake_difs"][seq])
        general_log2_difs["lake_difs"][seq].append("control")

for seq in general_log2_difs["ardley_difs"].keys():
    interest_data = ardley_data.loc[ardley_data["Assay"] == seq]
    dif = interest_data.loc[interest_data["type_g"] == "plastic"]["n_log2"].sum() - interest_data.loc[interest_data["type_g"] == "control"]["n_log2"].sum()
    if dif > 0:
        general_log2_difs["ardley_difs"][seq] = list(general_log2_difs["ardley_difs"][seq])
        general_log2_difs["ardley_difs"][seq].append("plastic")
    else:
        general_log2_difs["ardley_difs"][seq] = list(general_log2_difs["ardley_difs"][seq])
        general_log2_difs["ardley_difs"][seq].append("control")
        
import json

with open("../data/resistome_data/metadata/shared_different_seqs_per_place.json", "w") as fp:
    json.dump(general_log2_difs, fp)

Se da la mala suerte de que para aquellos resultados de T-test normales o mannwhitneys estoy perdiendo info de de dónde vienen (de qué test vienen esos resultados). Pero bueno, un poco lo que hay. Si viera a ojo que todo son plásticos puedo simplemente quedarme con los resultados del paso anterior y dejar ese print en el código como prueba.

NOTA: es el caso, asi que vuelvo a correr los resultados normales y me quedo esos